# Inferencing a Time Series Model using Channel Independence PatchTST

<ul>
<li>Contributors: IBM AI Research team and IBM Research Technology Education team
<li>Contact for questions and technical support: IBM.Research.JupyterLab@ibm.com
<li>Provenance: IBM Research
<li>Version: 1.0.0
<li>Release date: 
<li>Compute requirements: 1 CPU
<li>Memory requirements: 16 GB
<li>Notebook set: Time Series Foundation Model
</ul>

# Summary

**Patch Time Series Transformer (PatchTST)** is a new method for long-term forecasting based on Transformer modeling. In PatchTST, a time series is segmented into subseries-level patches that are served as input tokens to Transformer. PatchTST was first proposed in 2023 in [this paper](https://arxiv.org/pdf/2211.14730.pdf). It can achieve state-of-the-art results when compared to other Transformer-based models.

**Channel Independence PatchTST** is a variant of PatchTST where each channel contains a single univariate time series that shares the same embedding and Transformer weights across all the series.

This notebook is the last of three notebooks that should be run in sequence. After running the first notebook, `01_patch_tst_pretrain.ipynb`, a pretrained model was saved in your private storage. The second notebook, `02_patch_tst_fine_tune.ipynb`, loaded the pretrained model and created a fine tuned model, which was also saved in your private storage. This notebook demonstrates inferencing on test data using the fine tuned model. The goal of this demonstration is to forecast the future sensor values (load, oil temperature) of an electric transformer using test data from the ETTh1 benchmark dataset.

# Table of Contents

* <a href="#TST3_intro">Channel Independence PatchTST</a>
* <a href="#TST3_codes">Code Samples</a>
    * <a href="#TST3_import">Step 1. Imports</a>
    * <a href="#TST3_pipeln">Step 2. Load model and construct forecasting pipeline</a>
    * <a href="#TST3_datast">Step 3. Load and prepare datasets </a>
    * <a href="#TST3_forecs">Step 4. Generate forecasts </a>
    * <a href="#TST3_perfor">Step 5. Evaluate performance </a>
    * <a href="#TST3_visual">Step 6. Plot results </a>
* <a href="#TST3_concl">Conclusion</a>
* <a href="#TST3_learn">Learn More</a>

<a id="TST3_intro"></a>
# Channel Independence PatchTST

**Channel Independence PatchTST** is an efficient design of Transformer-based models for multivariate time series forecasting and self-supervised representation learning. It is demonstrated in the following diagram. It is based on two key components:

- segmentation of time series into subseries-level patches that are served as input tokens to Transformer

- channel independence where each channel contains a single univariate time series that shares the same embedding and Transformer weights across all the series.

Patching design naturally has three-fold benefit: local semantic information is retained in the embedding; computation and memory usage of the attention maps are quadratically reduced given the same look-back window; and the model can attend longer history.

Channel independence allows each time series to have its own embedding and attention maps while sharing the same model parameters across different channels.

<div> <img src="./data/figures/patchTST.png" alt="Drawing" style="width: 600px;"/></div>

<a id="TST3_codes"></a>
# Code Samples

This section includes documentation and code samples to demonstrate the use of the toolkit for inferencing.

<a id="TST3_import"></a>
## Step 1. Imports

In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import Image

from transformers.models.patchtst import PatchTSTForPrediction
from tsfmservices.toolkit.time_series_forecasting_pipeline import (
    TimeSeriesForecastingPipeline,
)
from tsfmservices.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfmservices.toolkit.util import select_by_index

# Customized IBM stylesheet for plots - dark background
# %run '/opt/ibm/visualization/plotly/plotly_template_dark.ipynb'

<a id="TST3_pipeln"></a>
## Step 2. Load model and construct forecasting pipeline

 Please adjust the following parameters to suit your application:
 - timestamp_column: column name containing timestamp information, use None if there is no such column
 - id_columns: List of column names specifying the IDs of different time series. If no ID column exists, use []
 - forecast_columns: List of columns to be modeled
 - finetuned_model_path: Path to the finetuned model
   

In [2]:
timestamp_column = "date"
id_columns = []
forecast_columns = ["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL", "OT"]
finetuned_model_path = "model/forecasting"

In [3]:
model = PatchTSTForPrediction.from_pretrained(finetuned_model_path)
# model.model.mask_input = False
forecast_pipeline = TimeSeriesForecastingPipeline(
    model=model,
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
)
context_length = model.config.context_length

tsp = TimeSeriesPreprocessor.from_pretrained("preprocessor")

<a id="TST3_datast"></a>
## Step 3. Load and prepare datasets

The specific data loaded here is Electricity Transformer Temperature (ETT) data - the temperature of the oil in an electric transformer. In the next cell, please adjust the following parameters to suit your application:

 - dataset_path: path to local .csv file, or web address to a csv file for the data of interest. Data is loaded with pandas, so anything supported by pd.read_csv is supported: (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
 - test_start_index, test_end_index: the start and end indices in the loaded data which delineate the test data.


In [4]:
dataset_path = (
    "https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh1.csv"
)
test_start_index=12 * 30 * 24 + 4 * 30 * 24 - context_length
test_end_index=12 * 30 * 24 + 8 * 30 * 24

In [5]:
data = pd.read_csv(
    dataset_path,
    parse_dates=[timestamp_column],
)

test_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=test_start_index,
    end_index=test_end_index,
)
test_data = tsp.preprocess(test_data)


In [6]:
test_data.head()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,__id
11008,2017-10-02 16:00:00,-0.247859,-1.191346,-0.250738,-0.534778,-0.128734,-1.976167,-0.724270,0
11009,2017-10-02 17:00:00,-0.005805,-0.966956,-0.025507,-0.184380,0.198567,-2.217347,-0.938949,0
11010,2017-10-02 18:00:00,-0.167174,-1.223402,-0.089833,-0.387351,-0.307528,-2.701292,-0.739636,0
11011,2017-10-02 19:00:00,-0.155648,-1.287514,-0.083310,-0.368664,-0.336839,-2.845682,-0.777995,0
11012,2017-10-02 20:00:00,-0.317017,-1.415737,-0.199277,-0.442896,-0.604542,-2.942471,-0.969571,0


<a id="TST3_forecs"></a>
## Step 4. Generate forecasts

 Note that the ouput will consist of a Pandas dataframe with the following structure.
 If you have specified timestamp and/or ID columns they will be included. The forecast
 columns will be named `{forecast column}_prediction`, for each `{forecast column}` that was
 specified.
 Each forecast column will be a vector of values with length equal to the prediction horizon
 that was specified when the model was trained.

In [7]:
forecasts = forecast_pipeline(test_data)
forecasts.head()

,date,HUFL_prediction,HULL_prediction,MUFL_prediction,MULL_prediction,LUFL_prediction,LULL_prediction,OT_prediction,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2017-10-23 23:00:00,"[0.37521761655807495, 0.3635786175727844, 0.22...","[0.3539564609527588, 0.4887229800224304, 0.455...","[0.5412428379058838, 0.562910795211792, 0.3786...","[0.345363587141037, 0.5174614191055298, 0.4958...","[-0.6434075832366943, -0.5244216322898865, -0....","[0.4739261865615845, 0.48465263843536377, 0.43...","[-0.7875686287879944, -0.8114785552024841, -0....","[0.35134100914001465, 0.06335341185331345, 0.1...","[0.6994680166244507, 0.05835173279047012, 0.09...","[0.4639110267162323, 0.21929237246513367, 0.25...","[0.5532732605934143, 0.1291615515947342, 0.202...","[-0.3964371979236603, -0.7540256977081299, -0....","[0.24680711328983307, 0.053229060024023056, 0....","[-0.8623406887054443, -0.869968831539154, -0.8..."
1,2017-10-24 00:00:00,"[0.20058709383010864, 0.01621261239051819, -0....","[0.46768373250961304, 0.6261210441589355, 0.40...","[0.313549280166626, 0.20299220085144043, 0.143...","[0.4207580089569092, 0.45269012451171875, 0.24...","[-0.5246557593345642, -0.7485214471817017, -0....","[0.3585163950920105, 0.36948254704475403, 0.30...","[-0.7825132608413696, -0.8246801495552063, -0....","[0.06335341185331345, 0.1438661515712738, 0.02...","[0.05835173279047012, 0.09040761739015579, 0.3...","[0.21929237246513367, 0.2513646185398102, 0.24...","[0.1291615515947342, 0.20287494361400604, 0.05...","[-0.7540256977081299, -0.5449438691139221, -0....","[0.053229060024023056, 0.24680711328983307, 0....","[-0.869968831539154, -0.8316100239753723, -0.8..."
2,2017-10-24 01:00:00,"[0.0014163851737976074, -0.1795271784067154, 0...","[0.40673455595970154, 0.621187686920166, 0.559...","[0.22471708059310913, 0.05083087086677551, 0.5...","[0.3357993960380554, 0.442648321390152, 0.4696...","[-0.764655351638794, -0.88056880235672, -0.622...","[0.06066074222326279, 0.12705408036708832, 0.0...","[-0.8221387267112732, -0.8062593936920166, -0....","[0.1438661515712738, 0.028774214908480644, 0.0...","[0.09040761739015579, 0.3468541204929352, -0.0...","[0.2513646185398102, 0.24502266943454742, 0.19...","[0.20287494361400604, 0.05544811859726906, 0.2...","[-0.5449438691139221, -0.7833362221717834, -0....","[0.24680711328983307, 0.29599499702453613, 0.3...","[-0.8316100239753723, -0.8775970935821533, -0...."
3,2017-10-24 02:00:00,"[0.1488836407661438, -0.030928760766983032, 0....","[0.24292689561843872, 0.303197979927063, 0.412...","[0.3364686369895935, 0.15613329410552979, 0.36...","[0.4205077290534973, 0.5495406985282898, 0.567...","[-0.9210672378540039, -0.7075734734535217, -0....","[-0.05519450455904007, 0.10066735744476318, 0....","[-0.78708416223526, -0.8099141716957092, -0.83...","[0.028774214908480644, 0.0862342119216919, 0.0...","[0.3468541204929352, -0.037815630435943604, 0....","[0.24502266943454742, 0.19990409910678864, 0.0...","[0.05544811859726906, 0.22156286239624023, -0....","[-0.7833362221717834, -0.45603516697883606, -0...","[0.29599499702453613, 0.34359613060951233, 0.1...","[-0.8775970935821533, -0.9159560203552246, -0...."
4,2017-10-24 03:00:00,"[0.060887306928634644, -0.030815094709396362, ...","[0.39665889739990234, 0.5518776178359985, 0.43...","[0.2469988465309143, 0.0821891725063324, 0.003...","[0.3083653748035431, 0.3329409956932068, 0.271...","[-0.7281441688537598, -0.4401995539665222, -0....","[-0.05365201458334923, 0.05062919855117798, -0...","[-0.7980622053146362, -0.8356679081916809, -0....","[0.0862342119216919, 0.0862342119216919, 0.293...","[-0.037815630435943604, 0.026295965537428856, ...","[0.19990409910678864, 0.058206457644701004, 0....","[0.22156286239624023, -0.16621123254299164, -0...","[-0.45603516697883606, -0.3075284957885742, 0....","[0.34359613060951233, 0.1992058902978897, 0.44...","[-0.9159560203552246, -0.8929623961448669, -0...."


<a id="TST3_perfor"></a>
## Step 5. Evaluate performance


In [ ]:
from tsevaluate.multivalue_timeseries_evaluator import CrossTimeSeriesEvaluator


labels_ = forecasts[id_columns + [timestamp_column] + forecast_columns]
forecasts_ = forecasts.drop(columns=forecast_columns)

eval = CrossTimeSeriesEvaluator(
    timestamp_column=timestamp_column,
    prediction_columns=[f"{c}_prediction" for c in forecast_columns],
    label_columns=forecast_columns,
    metrics_spec=["mse", "smape", "rmse", "mae"],
    multioutput="uniform_average"
)
eval.evaluate(labels_, forecasts_)


/Users/wmgifford/miniconda3/envs/test_ogv/lib/python3.10/site-packages/tsevaluate/multivalue_timeseries_evaluator.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[self.__class__._TMP_ID_COL] = self.__class__._TMP_ID_VAL


<a id="TST3_visual"></a>
## Step 6. Plot results



In [ ]:

plot_start = 2300
plot_stride = model.config.prediction_length
periodicity = "1h"
prediction_length = model.config.prediction_length
plot_index = range(plot_start, forecasts_.shape[0], plot_stride)

cols_to_plot = ["OT"]

fig = make_subplots(specs=[[{"secondary_y": True}]])
# for id in id_list:
#     data = train[train.id == id]
#     fig.add_trace(go.Scatter(x=data["date"], y=data["val"], name=f"{id}(train)"))
for c in cols_to_plot:
    fig.add_trace(
        go.Scatter(
            x=test_data[timestamp_column].iloc[plot_start:],
            y=test_data[c].iloc[plot_start:],
            name=f"{c}",
        )
    )

for c in cols_to_plot:
    forecast_name = f"{c}_prediction"
    for i in plot_index:
        start = forecasts_.loc[i, timestamp_column]

        timestamps = pd.date_range(
            start, freq=periodicity, periods=prediction_length + 1
        )
        timestamp = timestamps[1:]
        forecast_val = forecasts_.loc[i, forecast_name]

        fig.add_trace(go.Scatter(x=timestamps, y=forecast_val, name=forecast_name))

fig["layout"].update(height=600, width=900, title="graph", xaxis=dict(tickangle=-45))
Image(fig.to_image(format="png"))

<a id="TST3_concl"></a>
# Conclusion

This notebook showed how to perform inferencing in a Channel Independence PatchTST model. This is the last of three notebooks that were run in sequence using training and test data from the ETTh1 benchmark dataset, which represents oil temperature in an electric transformer.

The first notebook, `01_patch_tst_pretrain.ipynb`, created a pretrained model that was saved in your private storage. The second notebook, `02_patch_tst_fine_tune.ipynb`, loaded the pretrained model and created a fine tuned model, which was also saved in your private storage. This notebook performed inferencing on the fine tuned model. The goal of this demonstration was to forecast the future sensor values (load, oil temperature) of an electric transformer using test data.

<a id="TST3_learn"></a>
# Learn More

[This paper](https://arxiv.org/pdf/2211.14730.pdf) provides detailed information on Channel Independence PatchTST, including evaluations of its performance on 8 popular datasets, including Weather, Traffic, Electricity, ILI and 4 Electricity Transformer Temperature datasets (ETTh1, ETTh2, ETTm1, ETTm2). These publicly available datasets have been extensively utilized for benchmarking. We featured one of them (ETTh1) in this notebook.

If you have any questions or wish to schedule a technical deep dive, contact us by email at IBM.Research.JupyterLab@ibm.com.

© 2023 IBM Corporation